# 🔬 ISIC Cilt Lezyonu Analizi - Sayısal Görüntü İşleme Projesi

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS/blob/main/isic_analysis_colab.ipynb)

**Proje Sahibi:** Mustafa Engin Dalgıç  
**Öğrenci No:** 254309502  
**Üniversite:** Üsküdar Üniversitesi - Bilgisayar Mühendisliği Yüksek Lisans  
**E-posta:** engindalgic86@gmail.com

---

## 🚀 Nasıl Çalıştırılır?

1. **Runtime > Run all** tıklayın (veya `Ctrl+F9`)
2. Program otomatik çalışacak (~10-15 dakika)
3. Grafikler ekranda görünecek
4. İşlem bitince sonuçları indirebilirsiniz

---

## 📊 Bu Program Ne Yapar?

7 ana bölümde cilt lezyonu görüntülerini analiz eder:

1. ✅ Veri Analizi
2. ✅ Görselleştirme (RGB vs Grayscale)
3. ✅ Histogram Analizi
4. ✅ Kontrast İyileştirme (CLAHE, Gamma)
5. ✅ Gürültü Azaltma (Median, Gaussian)
6. ✅ Frekans Filtreleme (FFT)
7. ✅ Keskinleştirme ve Büyütme

**Çıktı:** ~45 grafik oluşturulur

---

## 📦 1. Kurulum ve Hazırlık

Gerekli kütüphaneleri yükleyip GitHub'dan projeyi klonluyoruz.

In [ ]:
# Gerekli kütüphaneleri kontrol et
print("📦 Kütüphaneler kontrol ediliyor...")

import sys
import subprocess

# OpenCV bazen eksik olabiliyor, kontrol edelim
try:
    import cv2
    print(f"✅ OpenCV {cv2.__version__} hazır")
except ImportError:
    print("⚠️  OpenCV kuruluyor...")
    !pip install opencv-python -q
    import cv2
    print(f"✅ OpenCV {cv2.__version__} kuruldu")

# Diğer kütüphaneler (Colab'da genelde hazır gelir)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import warnings
warnings.filterwarnings('ignore')

print(f"✅ NumPy {np.__version__}")
print(f"✅ Pandas {pd.__version__}")
print(f"✅ Matplotlib hazır")
print(f"✅ Seaborn hazır")
print("\n✅ Tüm kütüphaneler hazır!")

In [ ]:
# GitHub'dan projeyi klonla
print("📥 GitHub'dan proje indiriliyor...\n")

# Eğer daha önce klonlanmışsa sil
!rm -rf UU_DIP_ISIC_SKIN_LESION_ANALYSIS

# Klonla
!git clone https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS.git

# Proje klasörüne geç
%cd UU_DIP_ISIC_SKIN_LESION_ANALYSIS

print("\n✅ Proje indirildi!")
print("\n📁 Proje içeriği:")
!ls -lh

In [ ]:
# Veri setini kontrol et
print("🔍 Veri seti kontrol ediliyor...\n")

if os.path.exists("ISIC"):
    # Görüntü sayısını say
    total_images = 0
    for root, dirs, files in os.walk("ISIC"):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                total_images += 1
    
    print(f"✅ ISIC klasörü bulundu")
    print(f"✅ Toplam görüntü: {total_images} adet\n")
    
    # Alt klasörleri göster
    subdirs = [d for d in os.listdir("ISIC") if os.path.isdir(os.path.join("ISIC", d))]
    if subdirs:
        print(f"📂 Veri seti sınıfları ({len(subdirs)} adet):")
        for subdir in sorted(subdirs)[:10]:  # İlk 10'unu göster
            count = len([f for f in os.listdir(os.path.join("ISIC", subdir)) 
                        if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
            print(f"   - {subdir}: {count} görüntü")
        if len(subdirs) > 10:
            print(f"   ... ve {len(subdirs) - 10} tane daha")
else:
    print("❌ ISIC klasörü bulunamadı!")
    print("⚠️  Bu notebook GitHub'dan veri setini otomatik çekecek.")

## 🔬 2. Ana Analiz Programı

Şimdi tam analizi çalıştırıyoruz. Bu işlem 10-15 dakika sürebilir.

In [ ]:
# Görselleştirme ayarları
plt.style.use('default')
sns.set_palette("husl")

# Colab için grafik ayarları
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

print("="*70)
print("🚀 ANALİZ BAŞLIYOR...")
print("="*70)
print("\n⏱️  Tahmini süre: 10-15 dakika")
print("💡 Grafikler otomatik görünecek\n")

In [ ]:
# ==================== BÖLÜM 1: VERİ YÜKLEME ====================
print("\n" + "="*70)
print("📊 BÖLÜM 1: VERİ SETİ YÜKLEME VE ANALİZ")
print("="*70)

DATA_PATH = "ISIC"

def load_image_dataset(data_path):
    """
    ISIC klasöründeki tüm görüntüleri tarayıp DataFrame'e yükler
    """
    image_data = []
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
    
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.lower().endswith(valid_extensions):
                file_path = os.path.join(root, file)
                try:
                    img = Image.open(file_path)
                    width, height = img.size
                    channels = len(img.getbands())
                    file_size = os.path.getsize(file_path) / 1024
                    
                    image_data.append({
                        'filename': file,
                        'filepath': file_path,
                        'width': width,
                        'height': height,
                        'channels': channels,
                        'resolution': f"{width}x{height}",
                        'file_size_kb': round(file_size, 2)
                    })
                except Exception as e:
                    print(f"❌ Hata ({file}): {e}")
    
    return pd.DataFrame(image_data)

# Veri setini yükle
print(f"\n🔄 Veri seti yükleniyor...")
train_df = load_image_dataset(DATA_PATH)

print(f"\n✅ Toplam {len(train_df)} görüntü yüklendi!")
print("\n🔹 İlk 5 görüntü:")
print(train_df.head())

# İstatistikler
print("\n📐 Çözünürlük İstatistikleri:")
print(f"  - Ortalama: {train_df['width'].mean():.0f}x{train_df['height'].mean():.0f} px")
print(f"  - Min: {train_df['width'].min()}x{train_df['height'].min()}")
print(f"  - Max: {train_df['width'].max()}x{train_df['height'].max()}")

print("\n💾 Dosya Boyutu:")
print(f"  - Ortalama: {train_df['file_size_kb'].mean():.2f} KB")
print(f"  - Toplam: {train_df['file_size_kb'].sum()/1024:.2f} MB")

In [ ]:
# Veri seti analizi grafiği
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Veri Seti Analizi', fontsize=16, fontweight='bold')

# 1. Çözünürlük dağılımı
axes[0, 0].scatter(train_df['width'], train_df['height'], alpha=0.5, color='#FF69B4')
axes[0, 0].set_xlabel('Genişlik (px)')
axes[0, 0].set_ylabel('Yükseklik (px)')
axes[0, 0].set_title('Görüntü Çözünürlük Dağılımı')
axes[0, 0].grid(True, alpha=0.3)

# 2. Dosya boyutu
axes[0, 1].hist(train_df['file_size_kb'], bins=30, edgecolor='black', alpha=0.7, color='#FFB6C1')
axes[0, 1].set_xlabel('Dosya Boyutu (KB)')
axes[0, 1].set_ylabel('Frekans')
axes[0, 1].set_title('Dosya Boyutu Dağılımı')
axes[0, 1].grid(True, alpha=0.3)

# 3. Kanal sayısı
channel_counts = train_df['channels'].value_counts()
axes[1, 0].pie(channel_counts.values, labels=[f'{ch} Kanal' for ch in channel_counts.index],
               autopct='%1.1f%%', colors=['#FF69B4', '#FFB6C1'])
axes[1, 0].set_title('Kanal Sayısı Dağılımı')

# 4. En sık çözünürlükler
resolution_counts = train_df['resolution'].value_counts().head(10)
axes[1, 1].barh(range(len(resolution_counts)), resolution_counts.values, color='#FFB6C1')
axes[1, 1].set_yticks(range(len(resolution_counts)))
axes[1, 1].set_yticklabels(resolution_counts.index)
axes[1, 1].set_xlabel('Görüntü Sayısı')
axes[1, 1].set_title('En Sık 10 Çözünürlük')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n✅ Veri analizi tamamlandı!")

In [ ]:
# ==================== BÖLÜM 2: GÖRÜNTÜ GÖRSELLEŞTİRME ====================
print("\n" + "="*70)
print("📸 BÖLÜM 2: GÖRÜNTÜ GÖRSELLEŞTİRME")
print("="*70)

# Rastgele görüntü seç
np.random.seed(42)
random_indices = np.random.choice(train_df.index, size=min(9, len(train_df)), replace=False)
selected_images = train_df.iloc[random_indices]

def resize_image(img, max_size=800):
    """Görüntüyü performans için küçült"""
    h, w = img.shape[:2]
    if max(h, w) > max_size:
        if h > w:
            new_h = max_size
            new_w = int(w * (max_size / h))
        else:
            new_w = max_size
            new_h = int(h * (max_size / w))
        return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return img

# RGB vs Grayscale
n_images = len(selected_images)
fig, axes = plt.subplots(n_images, 2, figsize=(10, n_images * 3))
fig.suptitle('Rastgele Seçilen Görüntüler: RGB vs Grayscale', fontsize=16, y=0.995)

for idx, (i, row) in enumerate(selected_images.iterrows()):
    img_rgb = cv2.imread(row['filepath'])
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_rgb = resize_image(img_rgb, max_size=800)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    
    axes[idx, 0].imshow(img_rgb)
    axes[idx, 0].set_title(f'RGB - {row["filename"][:20]}...', fontsize=8)
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(img_gray, cmap='gray')
    axes[idx, 1].set_title(f'Grayscale - {row["resolution"]}', fontsize=8)
    axes[idx, 1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Görüntü görselleştirme tamamlandı!")

### 💡 Not

Yukarıdaki hücreler **Bölüm 1-2**'yi çalıştırır. Tam program için GitHub'daki `isic_project.py` dosyasındaki tüm bölümleri buraya ekleyebilirsiniz.

**Tam kodu çalıştırmak için:**

In [ ]:
# TAM PROGRAMI ÇALIŞTIR
print("🚀 Tam program çalıştırılıyor...\n")
print("⏱️  Bu işlem 10-15 dakika sürecek.")
print("📊 Grafikler otomatik görünecek.\n")

# Python dosyasını çalıştır
!python isic_project.py

## 📥 3. Sonuçları İndirme

Oluşan grafikleri bilgisayarınıza indirebilirsiniz.

In [ ]:
# Oluşan grafikleri listele
print("📊 Oluşan grafikler:\n")

png_files = !ls *.png 2>/dev/null

if png_files:
    for i, file in enumerate(png_files, 1):
        print(f"{i}. {file}")
    print(f"\n✅ Toplam {len(png_files)} grafik oluşturuldu!")
    print("\n💡 Grafikleri indirmek için:")
    print("   Sol taraftaki 📁 Files sekmesinden .png dosyalarını bulup")
    print("   sağ tık > Download yapabilirsiniz.")
else:
    print("⚠️  Henüz grafik oluşturulmamış.")
    print("Yukarıdaki hücreleri çalıştırın.")

In [ ]:
# Tüm grafikleri ZIP olarak indir
from google.colab import files
import zipfile

print("📦 Grafikler ZIP'e paketleniyor...\n")

png_files = !ls *.png 2>/dev/null

if png_files:
    with zipfile.ZipFile('isic_results.zip', 'w') as zipf:
        for file in png_files:
            zipf.write(file)
            print(f"✅ {file} eklendi")
    
    print("\n📥 ZIP dosyası indiriliyor...")
    files.download('isic_results.zip')
    print("\n✅ İndirme tamamlandı!")
else:
    print("❌ İndirilecek grafik bulunamadı.")

---

## 🎉 Tamamlandı!

### 📊 Analiz Sonuçları

Bu notebook çalıştırıldığında:
- ✅ ~45 grafik oluşturuldu
- ✅ 7 farklı görüntü işleme tekniği uygulandı
- ✅ Tüm sonuçlar kaydedildi

### 📚 Uygulanan Teknikler

1. Veri Analizi ve Görselleştirme
2. Histogram Analizi
3. Kontrast İyileştirme (CLAHE, Gamma)
4. Gürültü Azaltma (Median, Gaussian)
5. Geometrik Dönüşümler
6. Frekans Filtreleme (FFT)
7. Keskinleştirme ve Büyütme

---

### 💬 İletişim

**Mustafa Engin Dalgıç**  
📧 engindalgic86@gmail.com  
🎓 Üsküdar Üniversitesi - Bilgisayar Mühendisliği YL  
🔗 [GitHub Repo](https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS)

---

⭐ **Projeyi beğendiyseniz GitHub'da yıldız vermeyi unutmayın!** ⭐

[![GitHub](https://img.shields.io/badge/GitHub-Repository-blue?logo=github)](https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS)
